In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

4.1

In [ ]:
I = cv2.imread('I.jpg')
I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
J = cv2.imread('J.jpg')
J = cv2.cvtColor(J, cv2.COLOR_BGR2GRAY)

difference = cv2.absdiff(I, J)
plt.subplots(1, 3, figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(I, 'gray')
plt.axis('off')
plt.subplot(1, 3, 2)
plt.imshow(J, 'gray')
plt.axis('off')
plt.subplot(1, 3, 3)
plt.imshow(difference, 'gray')
plt.axis('off')
plt.show()
def block_optical_flow(I, J, W2, dX, dY):
    u = np.zeros(I.shape).astype(np.float32)
    v = np.zeros(I.shape).astype(np.float32)

    for j in range(W2 * 2, I.shape[0] - W2 * 2):
        for i in range(W2 * 2, I.shape[1] - W2 * 2):
            IO = I[j - W2:j + W2 + 1, i - W2:i + W2 + 1].astype(np.float32)

            x, y = 0 ,0
            most_similar = -1

            for l in range(-dX, dX + 1):
                for k in range(-dY, dY + 1):
                    JO = J[j + l - W2:j + l + W2 + 1, i + k - W2:i + k + W2 + 1].astype(np.float32)
                    dist = np.sum(np.square(IO - JO))
                    if most_similar == -1 or dist < most_similar:
                        most_similar = dist
                        x = k
                        y = l
                u[j, i] = x
                v[j, i] = y
    return u, v
def vis_flow(u, v):
    magnitude, angle = cv2.cartToPolar(u.astype(np.float32), v.astype(np.float32))
    magnitude_normalized = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

    HSV = np.zeros((u.shape[0], u.shape[1], 3), dtype=np.uint8)
    HSV[:,:,0] = (angle * 90 / np.pi).astype(np.uint8)
    HSV[:,:,1] = magnitude_normalized.astype(np.uint8)
    HSV[:,:,2] = 255

    RGB = cv2.cvtColor(HSV, cv2.COLOR_HSV2RGB)

    plt.imshow(RGB)
    plt.show()
W2 = dX = dY = 3
u, v = block_optical_flow(I, J, W2=5, dX=5, dY=5)

vis_flow(u, v)
I2 = cv2.imread('cm1.png')
I2 = cv2.cvtColor(I2, cv2.COLOR_BGR2GRAY)
J2 = cv2.imread('cm2.png')
J2 = cv2.cvtColor(J2, cv2.COLOR_BGR2GRAY)
W2 = dX = dY = 3
u2, v2 = block_optical_flow(I2, J2, W2, dX, dY)

vis_flow(u2, v2)

4.2

In [ ]:
def pyramid(im, max_scale):
    images =[im]
    for k in range(1, max_scale):
        images.append(cv2.resize(images[k - 1], (0, 0), fx=0.5, fy=0.5))
    return images
def pyramid_optical_flow(I, J, W2, dX, dY, max_scale):
    images_I = list(reversed(pyramid(I, max_scale)))
    images_J = list(reversed(pyramid(J, max_scale)))

    I = images_I[0]

    u_total = np.zeros(I.shape).astype(np.float32)
    v_total = np.zeros(I.shape).astype(np.float32)

    for I_list, J_list in zip(images_I, images_J):
        w, h = I_list.shape

        I = I_list.copy()

        for x in range(I_list.shape[0]):
            for y in range(I_list.shape[1]):
                new_x = np.clip(round(x - u_total[x, y]), 0, w - 1)
                new_y = np.clip(round(y - v_total[x, y]), 0, h - 1)

                I[x, y] = I_list[new_x, new_y]
            
        u, v = block_optical_flow(I, J_list, W2, dX, dY)

        u_total += u
        v_total += v

        u_total = cv2.resize(u_total, (0, 0), fx=2, fy=2, interpolation=cv2.INTER_LINEAR) * 2
        v_total = cv2.resize(v_total, (0, 0), fx=2, fy=2, interpolation=cv2.INTER_LINEAR) * 2

    return u_total, v_total
u3, v3 = pyramid_optical_flow(I, J, W2, dX, dY, 4)

vis_flow(u3, v3)
u4, v4 = pyramid_optical_flow(I2, J2, W2, dX, dY, 4)

vis_flow(u4, v4)
